In [1]:
import pandas as pd

#convert pandas series from German number format (1.000,5) to python number format (1000.5)
def make_column_numeric(se):
    se = se.str.replace('.','',regex=True)
    se = se.str.replace(',','.',regex=True)
    se = pd.to_numeric(se)
    return se

#INKAR data from bbsr
#https://www.inkar.de
df = pd.read_csv("data/inkar_commuting.csv",delimiter=";", header=[0,1])
df.columns = df.columns.map('_'.join)

#rename unnamed headers
df.rename({"Kennziffer_Unnamed: 0_level_1": "Kennziffer","Raumeinheit_Unnamed: 1_level_1":"Raumeinheit","Aggregat_Unnamed: 2_level_1":"Aggregated"}, axis=1,inplace=True)


years = ["2016","2017","2018","2019"]
for year in years:
    sv = "sozialversicherungspflichtig Beschäftigte am Wohnort_" + str(year)
    df[sv] = make_column_numeric(df[sv])

    
    df["Einpendler_" + str(year)] = make_column_numeric(df["Einpendler_" + str(year)])
    df["Auspendler_" + str(year)] = make_column_numeric(df["Auspendler_" + str(year)])
    df["Pendlersaldo_" + str(year)] = make_column_numeric(df["Pendlersaldo_" + str(year)])

    df["Pendler mit Arbeitsweg 50 km und mehr_" + str(year)] = make_column_numeric(df["Pendler mit Arbeitsweg 50 km und mehr_" + str(year)])
    df["Pendler mit Arbeitsweg 150 km und mehr_" + str(year)] = make_column_numeric(df["Pendler mit Arbeitsweg 150 km und mehr_" + str(year)])
    df["Pendler mit Arbeitsweg 300 km und mehr_" + str(year)] = make_column_numeric(df["Pendler mit Arbeitsweg 300 km und mehr_" + str(year)])

    df["pendler_50km_gesamt_" + str(year)] = df["Pendler mit Arbeitsweg 50 km und mehr_" + str(year)] *  df[sv]
    df["pendler_150km_gesamt_" + str(year)] = df["Pendler mit Arbeitsweg 150 km und mehr_" + str(year)] *  df[sv]
    df["pendler_300km_gesamt_" + str(year)] = df["Pendler mit Arbeitsweg 300 km und mehr_" + str(year)] *  df[sv]

df.rename({"Kennziffer":"AGS"},axis=1,inplace=True)

# Load geo coordinates of centroids of kreise (made with qgis)
geo = pd.read_csv("../general_data/kreise_geocoordinates.csv")
geo = geo.filter(['AGS','xcoord','ycoord','GEN'])

#merge on AGS
df = df.merge(geo, on="AGS",how="inner")

#save
df.to_csv("data/inkar_commuting_output.csv")
df



,AGS,Raumeinheit,Aggregated,Einpendler_1997,Einpendler_1998,Einpendler_1999,Einpendler_2000,Einpendler_2001,Einpendler_2002,Einpendler_2003,...,pendler_300km_gesamt_2017,pendler_50km_gesamt_2018,pendler_150km_gesamt_2018,pendler_300km_gesamt_2018,pendler_50km_gesamt_2019,pendler_150km_gesamt_2019,pendler_300km_gesamt_2019,xcoord,ycoord,GEN
0,1001,"Flensburg, Stadt",kreisfreie Stadt,"47,48","48,25","48,87","49,67","49,74","49,87","50,21",...,112175.45,446649.30,256357.10,128020.50,463188.59,270220.28,147716.11,9.438752,54.785155,Flensburg
1,1002,"Kiel, Stadt",kreisfreie Stadt,"43,48","44,44","44,61","45,15","45,98","46,23","46,97",...,298002.36,1204712.08,420248.40,316316.00,1188370.44,418181.40,315244.44,10.131725,54.325195,Kiel
2,1003,"Lübeck, Stadt",kreisfreie Stadt,"36,83","37,68","38,79","39,64","40,38","40,66","41,36",...,230965.84,1182692.72,308322.30,235589.86,1181316.60,318539.30,250509.55,10.727704,53.873031,Lübeck
3,1004,"Neumünster, Stadt",kreisfreie Stadt,"38,83","39,63","40,66","42,18","44,00","44,83","46,16",...,103761.84,432184.97,126407.52,104754.38,439419.60,131165.55,110155.05,9.983916,54.081674,Neumünster
4,1051,Dithmarschen,Landkreis,"11,72","11,63","11,98","12,64","14,89","12,80","12,73",...,145986.33,799231.44,188193.88,142212.00,815608.64,196115.92,150371.76,9.107561,54.134110,Dithmarschen
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
396,16073,Saalfeld-Rudolstadt,Landkreis,"11,48","12,06","11,80","12,89","13,13","13,39","14,00",...,123556.88,465587.33,206411.79,118190.80,443376.43,194563.53,116248.65,11.312354,50.640692,Saalfeld-Rudolstadt
397,16074,Saale-Holzland-Kreis,Landkreis,"26,47","26,89","28,44","29,41","30,08","31,78","33,35",...,124087.92,391555.65,193896.99,121741.32,380363.70,190181.85,117878.48,11.731716,50.904081,Saale-Holzland-Kreis
398,16075,Saale-Orla-Kreis,Landkreis,"15,83","16,53","18,19","18,97","19,92","18,89","19,69",...,93379.20,362650.08,152462.44,91681.20,357119.29,150187.18,88345.40,11.711224,50.580767,Saale-Orla-Kreis
399,16076,Greiz,Landkreis,"29,88","29,75","29,83","29,40","29,97","29,59","30,11",...,132436.02,463841.40,219653.82,130335.60,453674.04,212196.24,122830.44,12.074323,50.748627,Greiz


In [2]:
hamburg = df.copy()
hamburg = hamburg[hamburg["AGS"].isin([2000,3359,3353,1053,1062,1060,1056])]
hamburg.to_csv("data/communiting_hamburg.csv")
hamburg

,AGS,Raumeinheit,Aggregated,Einpendler_1997,Einpendler_1998,Einpendler_1999,Einpendler_2000,Einpendler_2001,Einpendler_2002,Einpendler_2003,...,pendler_300km_gesamt_2017,pendler_50km_gesamt_2018,pendler_150km_gesamt_2018,pendler_300km_gesamt_2018,pendler_50km_gesamt_2019,pendler_150km_gesamt_2019,pendler_300km_gesamt_2019,xcoord,ycoord,GEN
5,1053,Herzogtum Lauenburg,Landkreis,"31,63","32,56","33,50","34,76","35,87","35,82","36,38",...,213355.80,1152574.48,314675.84,221905.84,1159388.16,318529.82,221914.14,10.602534,53.589533,Herzogtum Lauenburg
8,1056,Pinneberg,Landkreis,"30,52","31,50","33,05","34,92","35,10","34,60","35,07",...,407823.90,960465.20,543421.10,436000.65,984302.34,558970.65,452316.48,9.735885,53.719215,Pinneberg
12,1060,Segeberg,Landkreis,"37,59","37,65","38,37","39,44","40,07","39,97","40,32",...,375866.72,1343271.72,490981.33,396774.98,1396813.96,508241.06,416553.92,10.141374,53.920087,Segeberg
14,1062,Stormarn,Landkreis,"48,38","49,57","49,85","50,98","51,48","51,39","52,02",...,364608.70,1101496.50,490495.45,376580.00,1117396.80,506171.20,387746.24,10.331402,53.720971,Stormarn
15,2000,"Hamburg, Stadt",kreisfreie Stadt,"34,51","34,88","34,89","35,61","35,84","35,73","36,03",...,3830927.32,6756440.02,4838869.46,4022403.87,6924824.71,5003397.21,4181026.24,10.018949,53.546596,Hamburg
35,3353,Harburg,Landkreis,"32,15","32,99","33,77","34,93","35,74","35,95","36,66",...,393541.68,1106524.93,539570.36,410763.15,1122250.40,564236.80,430438.00,9.963550,53.315626,Harburg
41,3359,Stade,Landkreis,"20,13","20,66","21,38","21,63","21,95","22,23","22,96",...,239434.66,964504.71,345216.69,251434.17,981735.22,355941.10,256608.70,9.416825,53.577931,Stade
